In [3]:
import pandas as pd
import numpy as np
import math

In [12]:
# Creation of the dataframe and Dropping the useless features
to_open = "Corr-20240413_TheBlaze.csv"
output_name = to_open.split('-')[1]
date = output_name.split('_')[0][:4] + "-" + output_name.split('_')[0][4:6] + "-" + output_name.split('_')[0][6:8]
print(date)
df = pd.read_csv(f"weekly_corrected//{to_open}")
df

2024-04-13


,Unnamed: 0,time,number,latitude,longitude,cape,sp,tcw,sshf,slhf,...,skt,cin,sm,st,sd,sf,tcc,tp,mx2t6,mn2t6
0,0,2024-04-13 00:00:00,1,50.75,4.25,0.000008,102003.984,15.363960,-122.0,274.0,...,284.12036,NaN,369.18323,287.43400,-2.384186e-07,-4.768372e-07,74.319050,-0.000002,NaN,NaN
1,1,2024-04-13 00:00:00,2,50.75,4.25,0.000008,102012.950,15.071946,-122.0,274.0,...,284.23294,NaN,363.34302,287.19153,-2.384186e-07,-4.768372e-07,95.964860,-0.000002,NaN,NaN
2,2,2024-04-13 00:00:00,3,50.75,4.25,0.000008,101973.250,16.052011,-122.0,274.0,...,284.30590,NaN,363.98376,287.16595,-2.384186e-07,-4.768372e-07,81.863420,-0.000002,NaN,NaN
3,3,2024-04-13 00:00:00,4,50.75,4.25,0.000008,102010.195,15.531214,-122.0,274.0,...,283.66342,NaN,369.52734,287.17917,-2.384186e-07,-4.768372e-07,74.905020,-0.000002,NaN,NaN
4,4,2024-04-13 00:00:00,5,50.75,4.25,0.000008,102014.830,17.011000,-122.0,274.0,...,284.09784,NaN,363.22437,287.23690,-2.384186e-07,-4.768372e-07,84.733765,-0.000002,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1445,1445,2024-04-20 00:00:00,46,50.75,4.25,17.571442,100912.330,16.865164,-7578484.0,-42831040.0,...,279.91640,NaN,374.47050,281.08185,1.188278e-03,1.654948e+00,99.178540,22.629534,282.78970,280.15463
1446,1446,2024-04-20 00:00:00,47,50.75,4.25,12.074219,101025.310,14.230239,-9091168.0,-40881760.0,...,280.74777,-0.413742,379.55130,281.06268,-2.384186e-07,1.285977e-01,64.380450,19.956219,281.26710,280.16520
1447,1447,2024-04-20 00:00:00,48,50.75,4.25,7.718903,101408.120,16.528276,-11803198.0,-44197416.0,...,279.90515,NaN,372.35843,281.51306,2.398515e-02,7.920194e-01,99.999510,18.431170,281.79060,277.94778
1448,1448,2024-04-20 00:00:00,49,50.75,4.25,10.898644,99728.950,8.490244,-11470323.0,-45636320.0,...,273.03534,NaN,384.80060,278.16720,2.176182e+00,3.821404e+00,76.443190,26.042990,275.51785,272.87677


In [13]:
df=df[['time','number','t2m']]

In [14]:
df

,time,number,t2m
0,2024-04-13 00:00:00,1,286.35
1,2024-04-13 00:00:00,2,286.35
2,2024-04-13 00:00:00,3,286.35
3,2024-04-13 00:00:00,4,286.35
4,2024-04-13 00:00:00,5,286.35
...,...,...,...
1445,2024-04-20 00:00:00,46,286.35
1446,2024-04-20 00:00:00,47,286.35
1447,2024-04-20 00:00:00,48,286.35
1448,2024-04-20 00:00:00,49,286.35


In [15]:
df['t2m']=df['t2m']-273.15

/var/folders/lw/bnynzkxj0cq2wnbmqw0rvyzh0000gn/T/ipykernel_27372/836502028.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['t2m']=df['t2m']-273.15


In [16]:
# Creation of the results Dataframes which will be made into one df later on
result_t2m  = pd.DataFrame(columns=['forecast_date','target','horizon','q0.025','q0.25','q0.5','q0.75','q0.975'])

In [17]:
times=df.time.unique()
number_of_hours=0
#defined earlier
#date = "2023-03-15"
row_index=0
for t in times:
  df_=df[df['time']==t]
  t2m = []
  t2m=df_.t2m.values


  horizon = str(number_of_hours)+' hour'
  result_t2m.loc[row_index]  = [date, 't2m', horizon , np.quantile(t2m,0.025),np.quantile(t2m,0.25),np.quantile(t2m,0.5),np.quantile(t2m,0.75),np.quantile(t2m,0.975)]
  row_index=row_index+1
  number_of_hours = number_of_hours + 6


In [10]:
# Dataframes concatination and result exportation
result_t2m = pd.DataFrame.to_csv(result_t2m,f"weekly_corrected/{output_name}", index=False)

In [10]:
result_t2m